# Домашнее задание 4

## Глубинное обучение в анализе графовых данных, ПМИ ВШЭ

In [2]:
import torch

!pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

Defaulting to user installation because normal site-packages is not writeable
Looking in links: https://data.pyg.org/whl/torch-2.1.2+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 593.9 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [46 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.linux-x86_64-cpython-311
      creating build/lib.linux-x86_64-cpython-311/torch_scatter
      copying torch_scatter/__init__.py -> build/lib.linux-x86_64-cpython-311/torch_scatter
      copying torch_scatter/placeholder.py -> build/lib.linux-x86_64-cpython-311/torch_scatter
      copying torch_scatter/scatter.py -> build/lib.linux-x86_64-cpython-311/torch_scatter
      copying torch_scatter/segment_coo.py -> build/lib.linux-x86_64-cpython-311/torch_scatter
     

## 1. Реализация TransE (10 баллов)

В этом задании требуется реализовать пайплайн обучения эмбдеддингов графа знаний с помощью [TransE](https://proceedings.neurips.cc/paper/2013/file/1cecc7a77928ca8133fa24680a88d2f9-Paper.pdf) для задачи прогнозирования отсутствующих ребер на наборе данных [Freebase](https://paperswithcode.com/dataset/fb15k) (FB15k-237), а также реализовать саму модель.

In [3]:
import torch_geometric
from torch_geometric.datasets.rel_link_pred_dataset import RelLinkPredDataset


dataset = RelLinkPredDataset('data', 'FB15k-237')
data = dataset[0]

Processing...
Done!


#### TransE
Ребра в графе знаний представляются тройками $(h, r, t)$. В TransE мы моделируем как объекты, так и отношения в пространстве эмбеддингов и пытаемся получить эмбеддинги, как $\textbf h + \textbf l \approx \textbf t$. Формально loss выглядит:

$$\sum_{((h, l, t), (h', l, t')) \in T_{batch}} [\gamma + d(\textbf{h} + \textbf{l}, \textbf t) - d(\textbf{h'} + \textbf l, \textbf{t'})]$$

где $(h', l, t')$ представляет собой тройку, заменяя head или tail случайным объектом.
$d(\boldsymbol{h}+\boldsymbol{l}, \boldsymbol{t})$ – показатель _различия_ положительного ребра. Кроме того, $d\left(\boldsymbol{h}^{\prime}+\boldsymbol{l}, \boldsymbol{t}^{\prime}\right)$ — это оценка _различия_ для отрицательной тройки, полученная изменением либо head, либо tail (но не оба) положительной тройки. Таким образом, TransE *предпочитает* более низкие оценки для положительных ребер и большие оценки для отрицательных ребер.

Что касается параметра $\gamma$, он используется для обеспечения того, чтобы оценка положительного ребра отличалась от оценки отрицательного ребра как минимум на $\gamma$.

Итого алгоритм TransE выглядит следующим образом:

![](https://production-media.paperswithcode.com/methods/Screen_Shot_2020-05-27_at_12.01.23_AM.png)

Что касается реализации модели, можно инициализировать $\textbf l$ и $\textbf e$ в соответствии с приведенным выше псевдокодом. Чтобы вычислить $d(\textbf{h} + \textbf{l}, \textbf t)$, нужно взять L2-норму $\textbf h + \textbf l - \textbf t$.

*Примечание: для повышения производительности можно нормализовать $\textbf e$ каждую эпоху, а не каждый мини-батч.*

__Вспомогательные функции:__

Одним из ключевых аспектов обучения модели является создание измененных троек путем замены head или tail случайным объектом:

In [ ]:
def create_neg_edge_index(edge_index, edge_type, num_entities):
    head_or_tail = torch.randint(high=2, size=edge_type.size(),
                                 device=device)
    rand_entities = torch.randint(high=num_entities,
                                  size=edge_type.size(), device=device)
    # change when 1, otherwise regular head
    heads = torch.where(head_or_tail == 1, rand_entities,
                        edge_index[0, :])
    # change when 0, otherwise regular tail
    tails = torch.where(head_or_tail == 0, rand_entities,
                        edge_index[1, :])
    return torch.stack([heads, tails], dim=0)

Оценивать качество будем по Hits@10, Mean Rank и MRR (mean reciprocal rank).

Hits@10 = $\frac{|\{r \in P | r \leq 10\}|}{|P|}$, где $|P|$ — количество оценок, а $r$ — ранг.

Mean Rank = $\frac{1}{|P|}\sum_{r \in P}r$

MMR = $\frac{1}{|P|}\sum_{r \in P}\frac{1}{r}$

Подробнее о метриках можно узнать [здесь](https://arxiv.org/pdf/2002.06914.pdf).

In [ ]:
def mrr(predictions, gt):
    indices = predictions.argsort()
    return (1.0 / (indices == gt).nonzero()[:, 1].float().add(1.0)).sum().item()


def mr(predictions, gt):
    indices = predictions.argsort()
    return ((indices == gt).nonzero()[:, 1].float().add(1.0)).sum().item()


def hit_at_k(predictions, gt, device, k=10):
    zero_tensor = torch.tensor([0], device=device)
    one_tensor = torch.tensor([1], device=device)
    _, indices = predictions.topk(k=k, largest=False)
    return torch.where(indices == gt, one_tensor, zero_tensor).sum().item()

__Требуется__ добиться качества хотя бы 0.17 MRR и 0.30 Hits@10

### 1.1 Вопрос о нормализации (2 балла)

Попробуйте обучить TransE без пятой строчки алгоритма (без нормализации по сущностям). Что происходит с обучением? Зачем требуется эта строка?

## 2. Нейросеть на гетерогенных данных (3 баллов)

Возьмите один из 2 датасетов (Freebase/ синтетический датасет hetero_graph далее)

In [ ]:
import numpy as np
import torch

n_users = 1000
n_items = 500
n_follows = 3000
n_clicks = 5000
n_dislikes = 500
n_hetero_features = 10
n_user_classes = 5
n_max_clicks = 10

follow_src = np.random.randint(0, n_users, n_follows)
follow_dst = np.random.randint(0, n_users, n_follows)
click_src = np.random.randint(0, n_users, n_clicks)
click_dst = np.random.randint(0, n_items, n_clicks)
dislike_src = np.random.randint(0, n_users, n_dislikes)
dislike_dst = np.random.randint(0, n_items, n_dislikes)

hetero_graph = dgl.heterograph({
    ('user', 'follow', 'user'): (follow_src, follow_dst),
    ('user', 'followed-by', 'user'): (follow_dst, follow_src),
    ('user', 'click', 'item'): (click_src, click_dst),
    ('item', 'clicked-by', 'user'): (click_dst, click_src),
    ('user', 'dislike', 'item'): (dislike_src, dislike_dst),
    ('item', 'disliked-by', 'user'): (dislike_dst, dislike_src)})

hetero_graph.nodes['user'].data['feature'] = torch.randn(n_users, n_hetero_features)
hetero_graph.nodes['item'].data['feature'] = torch.randn(n_items, n_hetero_features)
hetero_graph.nodes['user'].data['label'] = torch.randint(0, n_user_classes, (n_users,))
hetero_graph.edges['click'].data['label'] = torch.randint(1, n_max_clicks, (n_clicks,)).float()
# randomly generate training masks on user nodes and click edges
hetero_graph.nodes['user'].data['train_mask'] = torch.zeros(n_users, dtype=torch.bool).bernoulli(0.6)
hetero_graph.edges['click'].data['train_mask'] = torch.zeros(n_clicks, dtype=torch.bool).bernoulli(0.6)

Используя любую библиотеку (torch geometry, dgl, stellargraph) соберите нейронную сеть и обучите ее (решать задачу Node Classification) на одном из двух датасетов выше.

**БОНУСЫ:** 
(2 балла) Обучите нейросеть решать задачу link prediction

(1 балл) возьмите еще какой-нибудь гетерогенный датасет (не маленький и не синтетический) и обучите на нем

(4 балла) реализуйте самостоятельно Relational GCN и продемонстрируйте работоспобность вашего слоя. 